In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
import tqdm


# Paper recap

x1 and x2 are sequences generated from constrained random walks.  x1 (x2) changes with a prob per unit time of 
0.65 (0.99) by a random amount uniformly distributed between -0.5 and 0.5 (-1 and 1). The modulus of x is then 
taken to ensure a positive sequence. Initial v are randomly selected from a unif fistrbution between -1 and 1. 

v1 ans v2 are unif randomly selected between -1 and +1.

100 temporal patterns are simulated numerically with different sequences for x1 and x2, and different inital v1 and v2. Tau goes from 0.5 to 0.8 with dt=0.1

Network: 8 hidden nodes and trained on 50 of the processes. Results shown on 50 of the processes not seen during the training. Set alpha=0.01 and beta=40 000. 

Temporak pattern: network produces sequence v(1) ... v(T) given initial conditions v(0) and ext input x(0), ..., x(T-1) and time steps. 

The neural network takes as input v(i) and x(i), and computes the function dv/dt=F(v,x).
First layer: inputs v(i) and x(i)
Second layer: 8 hidden nodes with tanh function 
Output layr: dv/dt no activation function 

# Generate the data 

### The diff equation

In [108]:
def diff_equ(t, V, x1,x2):
    #x1, x2 =tf.split(X[tf.cast(t, tf.int32)],1)
    v1, v2 = V 
    dv1= tf.cast(x1, tf.float64) -2*v1 + 8*v2 -tf.cast(x1, tf.float64)*v1
    dv2= tf.cast(x2, tf.float64) -5*v1 + v2 -tf.cast(x2, tf.float64)*v2
    dV=np.array([dv1, dv2])
    return dV   

### Generate x value sequences for 0... T-1

In [143]:
def gen_x(T):
    #initial values 
    x1=np.absolute(np.array([np.random.uniform(low=-0.5, high=0.5)]))
    x2=np.absolute(np.array([np.random.uniform(low=-1.0, high=1.0)]))
    #x1=np.absolute(np.array([[np.random.uniform(low=-0.5, high=0.5)]]))
    #x2=np.absolute(np.array([[np.random.uniform(low=-1.0, high=1.0)]]))
    for i in range(T): 
        #generate a bernouilli according to PD
        p1 = np.random.binomial(1,p=0.65)
        p2 = np.random.binomial(1,p=0.99)
        u1=np.absolute(np.array([np.random.uniform(low=-0.5, high=0.5)]))
        u2=np.absolute(np.array([np.random.uniform(low=-1.0, high=1.0)]))
        x1=np.append(x1, x1[-1]+p1*u1, axis=0)
        x2=np.append(x2, x2[-1]+p2*u2, axis=0)
    #X=np.hstack((x1,x2))
    return x1, x2

In [144]:
#Regularisation para see equ.(24)
alpha=0.01
#Loss function para´see equ.(10)
beta=40000

ODESolver = tfp.math.ode.DormandPrince()

# initial conditions and time grid
t_initial = 0 # initial time
t_final = 8 # final time
n_times = 80 # number of time steps to churn out solution for
times = np.linspace(t_initial, t_final, n_times).astype(np.float32) 

v1=np.array([[np.random.uniform(low=-1.0, high=1.0)]])
v2=np.array([[np.random.uniform(low=-1.0, high=1.0)]])
v_initial = np.array([v1, v2]).astype(np.float32) # initial state vector
x1, x2=gen_x(n_times)
# integrate the ODE
results = ODESolver.solve(diff_equ, # system of ODEs (gradient function)
                                   t_initial, # initial time
                                   v_initial, # initial state
                                   solution_times=times,
                                   constants={'x1': x1,'x2': x2 }) # time grid to spit out solutions for

# extract results for the state solutions v(t)
data = tf.cast(tf.stack(results.states, axis=-1), tf.float32)

InvalidArgumentError: cannot compute Sub as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:Sub]

array([[ 0.40017716,  0.83352447],
       [ 0.41357302,  1.22561547],
       [ 0.41357302,  2.18052766],
       [ 0.41357302,  3.08017581],
       [ 0.41357302,  3.33488022],
       [ 0.91122499,  3.48516615],
       [ 0.91122499,  3.55686108],
       [ 0.91122499,  3.5999878 ],
       [ 1.34796603,  4.52015351],
       [ 1.59981833,  4.67873253],
       [ 2.05922584,  4.8814554 ],
       [ 2.17837711,  5.59546551],
       [ 2.47187198,  6.58475464],
       [ 2.64097316,  7.15772229],
       [ 2.91951391,  7.77074589],
       [ 3.08188081,  8.17335497],
       [ 3.08188081,  8.26712869],
       [ 3.08188081,  8.57724742],
       [ 3.39790407,  9.48525215],
       [ 3.39790407,  9.71844318],
       [ 3.63710127,  9.7721518 ],
       [ 4.01972621, 10.54227248],
       [ 4.15626879, 11.18451888],
       [ 4.34210419, 11.88488083],
       [ 4.59986972, 12.86365679],
       [ 4.90056325, 13.06774342],
       [ 4.9437225 , 13.56267304],
       [ 4.9437225 , 13.75400455],
       [ 5.33344721,

In [60]:
def update(v, dv):
    dv1, dv2=dv
    new_v1=v1+dv1*dt
    new_v2=v2+dv2*dt
    V=np.array([new_v1, new_v2])
    return V

In [59]:
# weights and biases:
n_state = 2
n_ext=2
n_hidden = 8
W1 = tf.Variable(tf.random.normal([(n_state+n_ext), n_hidden], 0, 0.1),  trainable=True)
b1 = tf.Variable(tf.random.normal([n_hidden], 0, 0.1),  trainable=True)
W2 = tf.Variable(tf.random.normal([n_hidden, n_state], 0, 0.1),  trainable=True)
b2 = tf.Variable(tf.random.normal([n_state], 0, 0.1),  trainable=True)

def dvdt_nn(v):    
    #dvdt = tf.matmul(tf.tanh(tf.matmul(tf.expand_dims(v, axis=0), W1) + b1), W2) + b2
    output=tf.matmul(tf.expand_dims(v, axis=0), W1) + b1
    dvdt = tf.matmul(tf.math.tanh(output), W2) + b2   
    return tf.squeeze(dvdt, axis=0)

For a given epoch: 
    Error: e=v-T where T is the target 
    Total error: 1/2 \sum \beta(e)^2

In [ ]:
# choose optimizer
optimizer = tf.keras.optimizers.Adam(lr = 1e-3)

# training step: computes predictions, compute MSE between predictions and data, computes gradients and applies them to the parameters
def training_step():
    
    # start the gradient tape: this records all operations in a way that gradients can be taken
    with tf.GradientTape() as tape:
        
        # solve the ODE
        predictions = tf.stack(ODESolver.solve(dvdt_nn,
                                   t_initial,
                                   v_initial,
                                   solution_times=times).states, axis=-1)
        
        # calculate loss
        error=data-predictions 
        loss=0.5*beta*np.linalg.norm(error)**2       
        
    # calculate gradients
    gradients = tape.gradient(loss, tape.watched_variables())
    
    # make gradient step
    optimizer.apply_gradients(zip(gradients, tape.watched_variables()))   
    
    return loss

array([27, 36])